<a name='1'></a>
# 1 - Importing libraries

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, ZeroPadding2D, Activation, Input, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D
from tensorflow.keras.layers import Concatenate
from tensorflow.keras.layers import Lambda, Flatten, Dense
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.layers import Layer
from tensorflow.keras import backend as K
K.set_image_data_format('channels_last')
import os
import numpy as np
from numpy import genfromtxt
import PIL

%matplotlib inline
%load_ext autoreload
%autoreload 2

<a name='2'></a>
# 2 - Loading Pretrained Facenet Model

In [4]:
from tensorflow.keras.models import model_from_json
model = tf.keras.models.load_model('/Users/harshsingh/Desktop/projects/face/model')
FRmodel = model

In [4]:
print(model.inputs)
print(model.outputs)

[<KerasTensor: shape=(None, 160, 160, 3) dtype=float32 (created by layer 'input_1')>]
[<KerasTensor: shape=(None, 128) dtype=float32 (created by layer 'Bottleneck_BatchNorm')>]


<a name='4'></a>
# 4 - Function to preprocess images and predict them

In [5]:
def img_to_encoding(image_path, model):
    img = tf.keras.preprocessing.image.load_img(image_path, target_size=(160, 160))
    img = np.around(np.array(img) / 255.0, decimals=12)
    x_train = np.expand_dims(img, axis=0) # add a dimension of 1 as first dimension
    embedding = model.predict_on_batch(x_train)
    return embedding / np.linalg.norm(embedding, ord=2)

<a name='5'></a>
# 5 - Simulation of a Database

In [6]:
import os

database = {}
base_dir = "/Users/harshsingh/Desktop/projects/face/indian celebrities dataset/cropped_data"

for name in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, name)
    
    # Ensure the path is a directory
    if os.path.isdir(folder_path):
        database[name] = []  # Initialize an empty list for the name
        count = 0  # Reset count for each person
        
        for image in os.listdir(folder_path):
            if count == 3:  # Stop after 5 images
                break
            
            img_path = os.path.join(folder_path, image)
            
            # Ensure the path is a valid file
            if os.path.isfile(img_path):
                # Process the image to get its embedding
                embedding = img_to_encoding(img_path, FRmodel)
                database[name].append(embedding)  # Add embedding to the list
                count += 1  # Increment the count

print("Database created successfully!")


# database["Harshil"] = img_to_encoding("/Users/harshsingh/Desktop/face/images/Harshil/Harshil_closeup.jpeg", FRmodel)
# database["Harsh"] = img_to_encoding("/Users/harshsingh/Desktop/face/images/Harsh/Harsh_abhi.jpeg", FRmodel)
# database["Aryan"] = img_to_encoding("/Users/harshsingh/Desktop/face/images/Aryan/Aryan_closeup.jpeg", FRmodel)
# database["Sarang"] = img_to_encoding("/Users/harshsingh/Desktop/face/images/Sarang/Sarang_closeup.jpeg", FRmodel)
# database["Papa"] = img_to_encoding("/Users/harshsingh/Desktop/face/images/Papa/Papa_plane.jpeg", FRmodel)

Database created successfully!


In [6]:
len(database)

14

In [11]:
print(f"""database["abhay_deol"][0][0][0] and the number of entries are {len(database)}""")

database["abhay_deol"][0][0][0] and the number of entries are 14


Load the images of Danielle and Kian:

<a name='6'></a>
# 6 - Face Verification

In [7]:
def verify(image_path, identity, database, model):
    
    encoding = img_to_encoding(image_path, model)
    dist = 100
    for embeddings in database[identity]:
        dist = min(np.linalg.norm(tf.subtract(embeddings, encoding)), dist)
    if dist < 0.9:
        print("It's " + str(identity) + ", welcome in!")
        door_open = True
    else:
        print("It's not " + str(identity) + ", please go away")
        door_open = False
    print(dist)
    return dist, door_open

In [8]:
peoples = ["Harsh", "Harshil","Aryan", "Sarang", "Papa"] ### "Papa", "Saran", "Sarang"]

In [78]:
base_dir = "images"  # Path to the "images" folder
for person in peoples:
    person_folder = os.path.join(base_dir, person)
    
    # Check if the folder exists
    if os.path.exists(person_folder):
        # Iterate through each image in the person's folder
        for image_name in os.listdir(person_folder):
            image_path = os.path.join(person_folder, image_name)
            
            # Check if it's an image file (optional check based on file extension)
            if image_name.lower().endswith(('.png', '.jpg', '.jpeg')):
                # Call the verify function for each image
                verify(image_path, "Papa", database, FRmodel)
            else:
                print(f"Skipping non-image file: {image_name}")


It's not Papa, please go away
1.3494343
It's not Papa, please go away
1.3059747
It's not Papa, please go away
1.0922953
It's not Papa, please go away
1.179243
It's not Papa, please go away
1.3380356
It's not Papa, please go away
1.1703198
It's not Papa, please go away
1.3531189
It's not Papa, please go away
1.2834737
It's not Papa, please go away
1.382893
It's not Papa, please go away
1.4386061
It's not Papa, please go away
1.4384489
It's not Papa, please go away
1.4638108
It's not Papa, please go away
1.3239197
It's not Papa, please go away
1.3198936
It's not Papa, please go away
1.402802
It's not Papa, please go away
1.485149
It's not Papa, please go away
1.4150637
It's Papa, welcome in!
0.0
It's Papa, welcome in!
0.6383572
It's Papa, welcome in!
0.55083877
It's Papa, welcome in!
0.5994069


In [13]:
verify("/Users/harshsingh/Desktop/projects/face/indian celebrities dataset/cropped_data/alok nath/62749_kweylwdatv_1499690485.jpg_cropped.jpg", "alok nath", database, FRmodel)

It's alok nath, welcome in!
0.5377962


(0.5377962, True)

<a name='7'></a>
# 7 - Face Recognition

In [7]:
def who_is_it(image_path, database, model):
    
    encoding =  img_to_encoding(image_path, model)

    min_dist = 1000
    identity = None
    for (name, db_enc) in database.items():
        for embeddings in db_enc:
            dist = np.linalg.norm(tf.subtract(embeddings, encoding))

            if dist < min_dist:
                min_dist = dist
                identity = name

    if min_dist > 0.95:
        print("Not in the database.")
    else:
        print ("it's " + str(identity) + ", the distance is " + str(min_dist))

    return min_dist, identity

In [38]:
base = "/Users/harshsingh/Desktop/face/Indian_actors_faces/amitabh_bachchan"
count = 0
for images in os.listdir(base):
      x, y=who_is_it(os.path.join(base, images), database, FRmodel)
      if(x>0.95 or y!= "amitabh_bachchan"):
            count = count+1
print(count)

it's amitabh_bachchan, the distance is 0.0
it's amitabh_bachchan, the distance is 0.0
it's amitabh_bachchan, the distance is 0.0
it's amitabh_bachchan, the distance is 0.0
it's amitabh_bachchan, the distance is 0.0
it's amitabh_bachchan, the distance is 0.0
it's amitabh_bachchan, the distance is 0.0
it's amitabh_bachchan, the distance is 0.0
it's amitabh_bachchan, the distance is 0.0
it's amitabh_bachchan, the distance is 0.0
it's amitabh_bachchan, the distance is 0.0
it's amitabh_bachchan, the distance is 0.0
it's amitabh_bachchan, the distance is 0.0
it's amitabh_bachchan, the distance is 0.0
it's amitabh_bachchan, the distance is 0.0
it's amitabh_bachchan, the distance is 0.80792135
it's amitabh_bachchan, the distance is 0.5087117
it's amitabh_bachchan, the distance is 0.7683447
it's amitabh_bachchan, the distance is 0.5892964
it's amitabh_bachchan, the distance is 0.49335662
it's amitabh_bachchan, the distance is 0.5859018
it's amitabh_bachchan, the distance is 0.48762754
it's amit

In [9]:
from Data_pipeline.crop_image import Cropping
test_path = "/Users/harshsingh/Desktop/projects/face/test.png"
output_path = "/Users/harshsingh/Desktop/projects/face"
cropper = Cropping(test_path, output_path, "test", 0, 10)
test_path = cropper.crop_faces()

# Test 1 with Younes pictures
who_is_it(test_path, database, FRmodel)

I0000 00:00:1735060701.727488 2495722 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.3), renderer: Apple M3
W0000 00:00:1735060701.741831 2510333 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.


Cropped face saved at: /Users/harshsingh/Desktop/projects/face/test_cropped.jpg
Not in the database.


(0.9629221, 'jeetendra')

In [ ]:
for people in peoples:
      verify("/Users/harshsingh/Desktop/face/Indian_actors_faces/irrfan_khan/images (9).jpg", people, database, FRmodel)

NameError: name 'peoples' is not defined